In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from solvers.improvements_8x8 import solve_puzzle, SolverConfig

def reconstruct_from_board(puzzle_path, board, grid_size=8):
    """Reconstruct image from board dict."""
    img = cv2.imread(str(puzzle_path))
    h, w = img.shape[:2]
    ph, pw = h // grid_size, w // grid_size
    
    # Extract pieces
    pieces = {}
    for idx in range(grid_size * grid_size):
        r, c = idx // grid_size, idx % grid_size
        pieces[idx] = img[r*ph:(r+1)*ph, c*pw:(c+1)*pw].copy()
    
    # Reconstruct
    out = np.zeros((h, w, 3), dtype=np.uint8)
    for (row, col), pid in board.items():
        out[row*ph:(row+1)*ph, col*pw:(col+1)*pw] = pieces[pid]
    return cv2.cvtColor(out, cv2.COLOR_BGR2RGB)

def load_correct(image_id):
    path = Path(f'./Gravity Falls/correct/{image_id}.png')
    if not path.exists():
        return None
    img = cv2.imread(str(path))
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def images_match(img1, img2):
    if img1 is None or img2 is None:
        return False
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    f1 = img1.astype(np.float32).flatten()
    f2 = img2.astype(np.float32).flatten()
    f1_norm = f1 - np.mean(f1)
    f2_norm = f2 - np.mean(f2)
    ncc = np.dot(f1_norm, f2_norm) / (np.linalg.norm(f1_norm) * np.linalg.norm(f2_norm) + 1e-10)
    return ncc > 0.95

In [2]:
# Load first 50 puzzle images
puzzle_dir = Path('./Gravity Falls/puzzle_8x8')
image_files = sorted(puzzle_dir.glob('*.jpg'), key=lambda x: int(x.stem))[:50]
print(f'Testing {len(image_files)} puzzle images')

Testing 50 puzzle images


In [3]:
# Solve all puzzles and compare with ground truth
# NOTE: 8x8 solver is slow (~1-2 min per image)
results = []
correct_count = 0

config = SolverConfig(verbose=True)

for img_path in image_files:
    image_id = img_path.stem
    print(f'\n=== Solving {image_id} ===')
    
    # Solve using improvements_8x8
    result = solve_puzzle(str(img_path), config)
    board = result['board']
    arrangement = result['arrangement']
    score = result['score']
    
    # Reconstruct solved image
    solved_img = reconstruct_from_board(img_path, board)
    
    # Load ground truth and compare
    correct_img = load_correct(image_id)
    is_correct = images_match(solved_img, correct_img)
    
    if is_correct:
        correct_count += 1
    
    results.append({
        'id': image_id,
        'solved': solved_img,
        'correct': correct_img,
        'arrangement': arrangement,
        'score': score,
        'is_correct': is_correct
    })
    print(f"Result: {'✓ CORRECT' if is_correct else '✗ WRONG'} (score={score:.4f})")

accuracy = 100 * correct_count / len(results) if results else 0
print(f'\n\nFinal Accuracy: {correct_count}/{len(results)} = {accuracy:.1f}%')


=== Solving 0 ===
IMPROVED 8x8 PUZZLE SOLVER

[Phase 0] Loading and computing descriptors...
  Loaded 64 pieces (8x8)
  Computing edge descriptors...
  Border likelihood range: [24.21, 137260.02]
  Computing pairwise compatibility...

[Phase 1] Detecting confident pairs...
  Detected 0 locked pairs
  Created 64 superpieces
  Sizes: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]...

[Phase 2] Assembling puzzle...
  Hungarian row assembly...
    Phase 1: Building candidate rows...
    Got 5000 candidate rows
    Phase 2: Selecting best rows with Hungarian...
  Beam search (width=10000)...
    Row 1: 570000 -> 10000
    Row 2: 490000 -> 10000
    Row 3: 410000 -> 10000
    Row 4: 330000 -> 10000
    Row 5: 250000 -> 10000
    Row 6: 170000 -> 10000
    Row 7: 90000 -> 10000
    Row 8: 10000 -> 10000
  Hungarian score: 0.5900
  Assembly score: 0.5900

[Phase 3] Refining solution...

  Starting refinement pipeline...
  Local swap refinement (initial: 0.1237)...
    Pass 1: score=0.1225, improvements=1
   

KeyboardInterrupt: 

In [ ]:
# Show incorrect solutions side-by-side with ground truth
incorrect = [r for r in results if not r['is_correct']]
print(f'Incorrect: {len(incorrect)}')

if incorrect:
    cols = 4
    rows = (len(incorrect) + 1) // 2
    fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, res in enumerate(incorrect):
        row = i // 2
        col = (i % 2) * 2
        
        axes[row, col].imshow(res['solved'])
        axes[row, col].set_title(f"{res['id']} - SOLVED (score={res['score']:.3f})")
        axes[row, col].axis('off')
        
        axes[row, col+1].imshow(res['correct'])
        axes[row, col+1].set_title(f"{res['id']} - CORRECT")
        axes[row, col+1].axis('off')
    
    for j in range(len(incorrect), rows * 2):
        row = j // 2
        col = (j % 2) * 2
        axes[row, col].axis('off')
        axes[row, col+1].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print('All puzzles solved correctly!')

In [ ]:
# Show all solved images in a grid
cols = 10
rows = (len(results) + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(20, 2*rows))
axes = axes.flatten()

for i, res in enumerate(results):
    axes[i].imshow(res['solved'])
    color = 'green' if res['is_correct'] else 'red'
    axes[i].set_title(res['id'], color=color, fontsize=8)
    axes[i].axis('off')

for j in range(len(results), len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()